# Problema Abordado

# Implementação

## Importações

In [28]:
import torch
from torchvision import transforms, datasets
from torchvision import models
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datasets import load_dataset
from PIL import Image
import os
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import random_split


##Funções Auxiliares

In [20]:
def train_model(
    model,          # Seu modelo
    train_loader,   # DataLoader de treino
    criterion,      # Função de perda
    optimizer,      # Otimizador
    device,         # "cuda" ou "cpu"
    epochs=10       # Número de épocas
):
    model.to(device)
    model.train()  # Modo treino

    for epoch in range(epochs):
        epoch_loss = 0.0

        for images, captions in train_loader:

            images = images.to(device)
            captions = captions.to(device)


            optimizer.zero_grad()


            outputs = model(images, captions[:, :-1])
            loss = criterion(
                outputs.view(-1, outputs.size(-1)),
                captions[:, 1:].reshape(-1)
            )

            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        # Loss médio da época
        avg_loss = epoch_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224,224)), # Padrão dos modelos pretreinados do ImageNet
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485, 0.456, 0.406],  ## Normalizando dados no padrão do ImageNet
                         std=[0.229, 0.224, 0.225])
])

In [30]:
#Função para ajustar dimensões dos tensores do dataset

def collate_fn(batch):
    # Extrai cada campo do batch
    images = [item["image_tensor"] for item in batch]
    captions = [item["caption"] for item in batch]
    input_ids = [item["tokenized_caption"] for item in batch]
    attention_masks = [item["attention_mask"] for item in batch]
    images_orig = [item["image_original"] for item in batch]

    # Empilha as imagens (todas já têm o mesmo shape, então stack direto)
    image_tensor = torch.stack(images)

    # Faz padding nas sequências de texto
    input_ids_padded = pad_sequence(input_ids, batch_first=True, padding_value=50256)  # eos_token_id para GPT-2
    attention_mask_padded = pad_sequence(attention_masks, batch_first=True, padding_value=0)

    return {
        "image_tensor": image_tensor,                    # (B, 3, 224, 224)
        "caption": captions,                             # Lista de strings
        "tokenized_caption": input_ids_padded,           # (B, T)
        "attention_mask": attention_mask_padded,         # (B, T)
        "image_original": images_orig                    # Lista de PIL Images
    }


## Dados

###Carregando o dataset


In [12]:


class DeepFashionDataset(Dataset):
    def __init__(self, labelDataset, image_dir, transform=None, tokenizer=None):
        self.df = labelDataset.reset_index(drop=True)
        self.image_dir = image_dir
        self.transform = transform

        # Tokenizador padrão: GPT-2
        self.tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
        self.tokenizer.pad_token = self.tokenizer.eos_token  # GPT-2 não tem pad_token original

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        caption = row["caption"]
        img_path = os.path.join(self.image_dir, row["path"])

        # Imagem original e transformada
        image_pil = Image.open(img_path).convert("RGB")
        image_tensor = self.transform(image_pil) if self.transform else image_pil

        # Tokenização (sem truncamento, padding tratado no collate_fn no Dataloader)
        tokens = self.tokenizer(caption, return_tensors="pt")
        input_ids = tokens["input_ids"].squeeze(0)
        attention_mask = tokens["attention_mask"].squeeze(0)

        return {
            "image_tensor": image_tensor,
            "caption": caption,
            "tokenized_caption": input_ids,
            "attention_mask": attention_mask,
            "image_original": image_pil
        }


In [13]:
labels_df = pd.read_csv('datasets/labels_front.csv')
labels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12278 entries, 0 to 12277
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   image_id      12278 non-null  object
 1   caption       12278 non-null  object
 2   path          12278 non-null  object
 3   gender        12278 non-null  object
 4   product_type  12278 non-null  object
 5   product_id    12278 non-null  object
 6   image_type    12278 non-null  object
dtypes: object(7)
memory usage: 671.6+ KB


### Data Loader

In [14]:
dataset = DeepFashionDataset(
    labelDataset = labels_df,
    image_dir = "datasets/selected_images",
    transform = transform
)


In [31]:
train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])


batch_size = 32 #Quantos exemplos são processsados juntos durante o treino
#Como não estamos truncando o tamanho dos tokens precisamos garantir que todas as legendas
# tenham o mesmo tamanho de tensor o collate_fn ajustara esse tamanho com

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

## Rede Implementada

In [25]:
class ResNetEncoder(nn.Module): #herda da classe nn.Module do pythorch
    def __init__(self): ##inicializa a  classe resnetEncoder
        super().__init__() ##inicializa a classe pai nn.Module

        self.encoder = models.resnet50(pretrained=True) #Carrega resnet treinada com o imagenet. O pretreined=True garante que seja carregada com os pesos treinados com o imagenet
        self.encoder.fc = nn.Identity() #Modifica a resnet para que a ultima camada não seja uma camada de classificação mas sim uma camada de identidade
        #Não queremos classificar nada, apenas obter os pesos da rede para construir os "embeddings de imagem"

    def forward(self, imgTensor):
      return self.encoder(imgTensor) #Realiza foward na rede



In [32]:
class Gpt2Decoder(nn.Module):
  def __init__(self, image_dim=2048): #inicializa decoder
    super().__init__() ##inicializa classe pai nn.module

    self.gpt = GPT2LMHeadModel.from_pretrained("gpt2")
    self.image_proj = nn.Linear(image_dim, self.gpt.config.n_embd) #ajustaa a dimensão de saida da resnet para a dimensão do embedding do gpt

  def foward(self, textTokens, attention_mask, image_features):

    batch_size = textTokens.size(0)

    #Obtem embedding dos tokens e ajusta tamanho das imagens para produzir o input final
    img_embed = self.image_proj(image_features).unsqueeze(1)
    text_embeds = self.gpt.transformer.wte(textTokens)
    gpt_input = torch.cat([img_embed, text_embeds], dim=1)

    #Ajusta mascara de atenção pois agora temos a imagem concatenada
    prefix_mask = torch.ones(batch_size, 1, dtype=attention_mask.dtype, device=attention_mask.device)
    full_attention_mask = torch.cat([prefix_mask, attention_mask], dim=1)

    outputs = self.gpt(inputs_embeds=gpt_input, attention_mask=full_attention_mask)

    return outputs.logits


In [ ]:
class ImageCaptionModel(nn.Module):


# Treinamento da Rede

# Qualidade dos Resultados

# Discussão Geral